<a href="https://colab.research.google.com/github/JSJeong-me/Vision_tasks/blob/main/Object_Detection/yolo-cv-crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/ultralytics/yolov5/issues/36

In [ ]:
#pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
import torch
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:
img = cv2.imread('traffic-signal-1.jpg')[..., ::-1]

In [ ]:
cv2_imshow(img)

In [ ]:
model.classes = [2]

In [ ]:
model.conf = 0.6

In [ ]:
results = model(img, size = 720)

In [ ]:
results.save()

In [ ]:
results.crop(save=True)

In [ ]:
import torch
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Image
#img = './traffic-signal-1.jpg'
img = cv2.imread('./traffic-signal-1.jpg')[..., ::-1]  # OpenCV image (BGR to RGB)

img.shape

In [ ]:
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes =[2, 9]  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs

# Inference
results = model(img, size=920)

In [ ]:
from io import BytesIO
from PIL import Image
import base64

In [ ]:
results.imgs # array of original images (as np array) passed to model for inference
results.render()  # updates results.imgs with boxes and labels
for img in results.imgs:
    buffered = BytesIO()
    img_base64 = Image.fromarray(img)
    img_base64.save(buffered, format="JPEG")
    #print(base64.b64encode(buffered.getvalue()).decode('utf-8'))  # base64 encoded image with results

In [ ]:
img_base64

In [ ]:
crops = results.crop(save=True)  # cropped detections dictionary

In [ ]:
no_of_frames = 1  # no_of_frames

In [ ]:
for i in range(no_of_frames):
  print(results.pandas().xyxy[i])

In [ ]:
crops_image = results.pandas().xyxy[no_of_frames-1]

In [ ]:
len(crops_image)

In [ ]:
results.pandas().xyxy[no_of_frames-1]

In [ ]:
results.pandas().xyxy[no_of_frames-1].sort_values('ymin')  # sorted left-right

In [ ]:
target_signal = 4

In [ ]:
crops_image.iloc[target_signal,5] == 9 # Traffic light

In [ ]:
if (crops_image.iloc[target_signal, 5] == 9): # traffic light
  x1, y1, x2, y2 = int(crops_image.iloc[target_signal,0]), int(crops_image.iloc[target_signal,1]), int(crops_image.iloc[target_signal,2]), int(crops_image.iloc[target_signal,3])
  print(x1, y1, x2, y2)
  signal1 = img[y1:y2, x1:x2]
  

In [ ]:
cv2_imshow(signal1)
#cv2.imwrite('./signal-yellow.jpg', signal1)

In [ ]:
hsv_frame = cv2.cvtColor(signal1, cv2.COLOR_BGR2HSV)

# Red color
low_red = np.array([1, 220, 90]) # 161,155,84 
high_red = np.array([179, 255, 255])
red_mask = cv2.inRange(hsv_frame, low_red, high_red)
red = cv2.bitwise_and(signal1, signal1, mask=red_mask)

# Blue color
low_blue = np.array([94, 80, 2])
high_blue = np.array([126, 255, 255])
blue_mask = cv2.inRange(hsv_frame, low_blue, high_blue)
blue = cv2.bitwise_and(signal1, signal1, mask=blue_mask)

# Green color
low_green = np.array([25, 52, 72])
high_green = np.array([102, 255, 255])
green_mask = cv2.inRange(hsv_frame, low_green, high_green)
green = cv2.bitwise_and(signal1, signal1, mask=green_mask)

# Every color except white
low = np.array([0, 42, 0])
high = np.array([179, 255, 255])
mask = cv2.inRange(hsv_frame, low, high)
result = cv2.bitwise_and(signal1, signal1, mask=mask)

#cv2_imshow(signal1)
cv2_imshow(blue)
cv2_imshow(green)
cv2_imshow(red)
#cv2_imshow(result)

H :0  202
S :40  227
V :206  255

In [ ]:
l_h, u_h = 0,202
l_s, u_s = 40, 227
l_v, u_v = 206, 255

l_yellow = np.array([l_h, l_s, l_v])
u_yellow = np.array([u_h, u_s, u_v])

In [ ]:
yellow_mask = cv2.inRange(hsv_frame, l_yellow, u_yellow)
yellow = cv2.bitwise_and(signal1, signal1, mask=yellow_mask)

In [ ]:
cv2_imshow(yellow)

In [ ]:
blur = cv2.medianBlur(yellow,15)

In [ ]:
cv2_imshow(blur)

In [ ]:
kernel = np.ones((12,12),np.uint8)

In [ ]:
shaved = cv2.morphologyEx(blur,cv2.MORPH_OPEN,kernel)

In [ ]:
cv2_imshow(shaved)